In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

crédit et dépots

In [2]:
file_path = "PATH/data/Données crédtis &amp; dépôts 2022 2023 2024 cv.xlsx"
xls = pd.ExcelFile(file_path)
print(xls.sheet_names)

['Tableau3', 'CH01_20250416_110051']


In [3]:
df = pd.read_excel(file_path,sheet_name='Tableau3')
df.head()

,Année,CSP,ID,Code Produit Pilotage,Libellé Produit Pilotage,CODE_SSFAMILLE_PRODUIT,LIBELLÉ_SSFAMILLE_PRODUIT,Ligne Métier,Mois (#),Pays Client,Région,Secteur Activité BAM,PNB Vision Commerciale Cumulé,EM C,Poss_Crédit,Poss_Dépot
0,2022,AGENT GERANT IMMOB SYNDIC,221,10102,FACILITE DE CAISSE,10100,CREDIT A LA TRESORERIE,TPE (hors BP),12,MOROCCO,REGION CASABLANCA-SETTAT,BATIMENTS ET TRAVAUX PUBLICS,331.4892,-177.758618,Oui,Non
1,2022,AGENT GERANT IMMOB SYNDIC,221,20101,COMPTE CHEQUE,20100,DEPOTS A VUE,TPE (hors BP),12,MOROCCO,REGION CASABLANCA-SETTAT,BATIMENTS ET TRAVAUX PUBLICS,23.7405,953.341012,Non,Oui
2,2022,AGENT GERANT IMMOB SYNDIC,221,40101,AISANCE,40100,CARTES,TPE (hors BP),12,MOROCCO,REGION CASABLANCA-SETTAT,BATIMENTS ET TRAVAUX PUBLICS,66.5000,0.000000,Non,Non
3,2022,AGENT GERANT IMMOB SYNDIC,221,40126,CARTE WEB PAY,40100,CARTES,TPE (hors BP),12,MOROCCO,REGION CASABLANCA-SETTAT,BATIMENTS ET TRAVAUX PUBLICS,56.1526,0.000000,Non,Non
4,2022,AGENT GERANT IMMOB SYNDIC,221,50112,PACK HISSAB KAFY,50100,PACKAGES,TPE (hors BP),12,MOROCCO,REGION CASABLANCA-SETTAT,BATIMENTS ET TRAVAUX PUBLICS,359.1000,0.000000,Non,Non


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267372 entries, 0 to 267371
Data columns (total 16 columns):
Année                            267372 non-null int64
CSP                              267372 non-null object
ID                               267372 non-null int64
Code Produit Pilotage            267372 non-null object
Libellé Produit Pilotage         267372 non-null object
CODE_SSFAMILLE_PRODUIT           267372 non-null int64
LIBELLÉ_SSFAMILLE_PRODUIT        267372 non-null object
Ligne Métier                     267372 non-null object
Mois (#)                         267372 non-null int64
Pays Client                      267372 non-null object
Région                           267372 non-null object
Secteur Activité BAM             267372 non-null object
PNB Vision Commerciale Cumulé    267372 non-null float64
EM C                             267372 non-null float64
Poss_Crédit                      267372 non-null object
Poss_Dépot                       267372 non-null ob

In [5]:
# Fonctions de transformation
def a_package(libelles):
    return int((libelles == "PACKAGES").any())
 
def nb_types_credit(df_grp):
    credit_df = df_grp[df_grp["Poss_Crédit"] == "Oui"]
    return credit_df[["LIBELLÉ_SSFAMILLE_PRODUIT", "Libellé Produit Pilotage"]].drop_duplicates().shape[0]
 
def nb_depots(df_grp):
    return df_grp[df_grp["Poss_Dépot"] == "Oui"].shape[0]
 
def get_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None
 
# Liste des années et clients
all_years = sorted(df["Année"].dropna().unique())
all_ids = df["ID"].dropna().unique()
full_index = pd.MultiIndex.from_product([all_ids, all_years], names=["ID", "Année"])
df_full = pd.DataFrame(index=full_index).reset_index()
 
# Marquer la présence effective dans les données
presence_df = df[["ID", "Année"]].drop_duplicates()
presence_df["Est_actif"] = 1
df_full = df_full.merge(presence_df, on=["ID", "Année"], how="left")
df_full["Est_actif"] = df_full["Est_actif"].fillna(0).astype(int)
 
# Calcul des agrégats par ID et Année
agg_df = df.groupby(["ID", "Année"]).agg(
    A_un_crédit=('Poss_Crédit', lambda x: int((x == "Oui").any())),
    A_un_dépot=('Poss_Dépot', lambda x: int((x == "Oui").any())),
    PNB_total=('PNB Vision Commerciale Cumulé', 'sum'),
    Nb_produits_differents=('Libellé Produit Pilotage', pd.Series.nunique),
    Nb_ssfamilles=('LIBELLÉ_SSFAMILLE_PRODUIT', pd.Series.nunique),
    A_un_package=('LIBELLÉ_SSFAMILLE_PRODUIT', a_package)
).reset_index()
 
# Ajouter types de crédit et nombre de dépôts
grouped = df.groupby(["ID", "Année"])
types_credit = grouped.apply(nb_types_credit).reset_index(name="Nb_types_credit")
nb_depots = grouped.apply(nb_depots).reset_index(name="Nb_depots")
 
agg_df = agg_df.merge(types_credit, on=["ID", "Année"], how="left")
agg_df = agg_df.merge(nb_depots, on=["ID", "Année"], how="left")

# Calculer les encours moyens crédit et dépôt par ID et Année
encours_moyens = df.groupby(["ID", "Année"]).apply(
    lambda x: pd.Series({
        "EMC_credit": x.loc[x["Poss_Crédit"] == "Oui", "EM C"].sum(),
        "EMC_depot": x.loc[x["Poss_Dépot"] == "Oui", "EM C"].sum()
    })
).reset_index()
 
# Fusionner avec les autres agrégats
agg_df = agg_df.merge(encours_moyens, on=["ID", "Année"], how="left")
 
# Fusionner avec l’index complet
agg_df_full = df_full.merge(agg_df, on=["ID", "Année"], how="left")
 
# Remplir les NaN avec 0 (sauf pour variables catégorielles à venir)
cols_to_fill = [col for col in agg_df_full.columns if col not in ['ID', 'Année']]
agg_df_full[cols_to_fill] = agg_df_full[cols_to_fill].fillna(0)
 
# Pivot des features numériques
num_cols = ['A_un_crédit', 'A_un_dépot', 'PNB_total',  'Nb_produits_differents',
            'Nb_ssfamilles', 'A_un_package', 'Nb_types_credit', 'Nb_depots', 'Est_actif', 'EMC_credit', 'EMC_depot']
 
df_pivot = agg_df_full.pivot(index='ID', columns='Année', values=num_cols)
df_pivot.columns = [f"{col}_{year}" for col, year in df_pivot.columns]
df_pivot.reset_index(inplace=True)
 
# Variables statiques : on les récupère une seule fois par ID
static_cols = ['CSP', 'Ligne Métier', 'Pays Client', 'Région']
static_df = df.groupby("ID")[static_cols].agg(get_mode).reset_index()
 
# Fusion finale
df_last = df_pivot.merge(static_df, on="ID", how="left")


: 

: 

In [5]:
df_last.head()

,ID,A_un_crédit_2022,A_un_crédit_2023,A_un_crédit_2024,A_un_dépot_2022,A_un_dépot_2023,A_un_dépot_2024,PNB_total_2022,PNB_total_2023,PNB_total_2024,...,EMC_credit_2022,EMC_credit_2023,EMC_credit_2024,EMC_depot_2022,EMC_depot_2023,EMC_depot_2024,CSP,Ligne Métier,Pays Client,Région
0,1,0.0,1.0,1.0,1.0,1.0,1.0,300.6199,413.6433,-15.0822,...,0.000000,-255.822366,-452.466000,6.143569e+03,70.239559,7.980000,RENTIER,Particuliers & Professionnels (hors BP),MOROCCO,REGION ORIENTAL
1,2,0.0,0.0,0.0,1.0,1.0,1.0,425.8128,339.3229,322.5250,...,0.000000,0.000000,0.000000,2.192384e+03,1222.500691,1251.859339,EMPLOYE,MRE (hors BP),USA,REGION CASABLANCA-SETTAT
2,3,1.0,1.0,1.0,1.0,1.0,0.0,3546.1923,2295.9790,34.1677,...,-1685.083327,-1735.729399,-2101.359155,1.315480e+05,76947.919323,0.000000,ARTISANAT METIER INDEPEND,TPE (hors BP),MOROCCO,REG MKECH-BNIMELLAL-TAFIL
3,4,1.0,1.0,1.0,1.0,1.0,1.0,54095.4092,12147.7545,16879.9078,...,0.000000,-63.435862,0.000000,2.105308e+06,413071.165264,601432.884531,COMMERCE BOULANGERIE CAFE,TPE (hors BP),MOROCCO,REG TANGER-TETOUAN-HOCEIM
4,5,0.0,0.0,0.0,1.0,1.0,1.0,1626.8161,300.3805,685.3091,...,0.000000,0.000000,0.000000,5.291207e+04,364.180527,14847.971738,OUVRIER,MRE (hors BP),FRANCE,REGION SOUSS-MASSA-SAHARA


In [ ]:
df_last.info()

In [6]:
# Export final
df_last.to_excel("clients_features.xlsx", index=False)
print("Fichier exporté avec succès.")

Fichier exporté avec succès.


Nb Operations

In [9]:
file_path = "C:/Users/mohameds.berrada/Downloads/codespace/test_codespace/data/operations.xlsx"
xls = pd.ExcelFile(file_path)
print(xls.sheet_names)

['CH129_20250424_160650']


In [14]:
df = pd.read_excel(file_path,sheet_name='CH129_20250424_160650')
df.head()

,Année,Famille opération,Type opération,Région,Marché,Code Agence,Volume Opérations en Kdh,Nombre Opérations,ID,Num client BENEF stage
0,2021,OPERATIONS A L'INTERNATIONAL,CREDOC IMPORT,REGION CASABLANCA-SETTAT,PETITE MOYENNE ENTREPR,28.0,2.0,2,16329,-9
1,2021,OPERATIONS A L'INTERNATIONAL,CREDOC IMPORT,REGION CASABLANCA-SETTAT,PETITE MOYENNE ENTREPR,340.0,1609.6,15,14653,-9
2,2021,OPERATIONS A L'INTERNATIONAL,CREDOC IMPORT,REGION CASABLANCA-SETTAT,PETITE MOYENNE ENTREPR,1030.0,468.5,2,13876,-9
3,2021,OPERATIONS A L'INTERNATIONAL,RAPATRIEMENT,INDETERMINE,PETITE MOYENNE ENTREPR,225.0,8.9,1,-9,11298
4,2021,OPERATIONS A L'INTERNATIONAL,RAPATRIEMENT,INDETERMINE,PARTICULIER,605.0,147.3,12,-9,2363


In [15]:
import os
 
# Réduction des threads pour éviter les erreurs OpenBLAS
os.environ["OPENBLAS_NUM_THREADS"] = "1"
 
# Chargement des données
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
 
# Nettoyage / conversions
df["nombre_opérations"] = pd.to_numeric(df["nombre_opérations"], errors="coerce")
df["volume_opérations_en_kdh"] = pd.to_numeric(df["volume_opérations_en_kdh"], errors="coerce")
df["vers_autre_banque"] = df["num_client_benef_stage"].isin([-1, -9])
 
# 1. Total opérations + activité en 2023
agg_df = df.groupby(["id", "année"]).agg(
    total_operations=("nombre_opérations", "sum")
).reset_index()
active = df[df["année"] <= 2023][["id"]].drop_duplicates()
active["active"] = True
agg_df = agg_df.merge(active, on="id", how="left")
agg_df["active"] = agg_df["active"].fillna(False)
 
# 2. Top famille d'opérations
top_famille = (
    df.groupby(["id", "année", "famille_opération"])
    .agg(total_ops=("nombre_opérations", "sum"))
    .reset_index()
    .sort_values(["id", "année", "total_ops"], ascending=[True, True, False])
    .drop_duplicates(["id", "année"])
    .rename(columns={"famille_opération": "top_famille_opération", "total_ops": "top_famille_total_ops"})
)
 
# 3. Détails de la top famille
top_details = df.merge(
    top_famille[["id", "année", "top_famille_opération"]],
    left_on=["id", "année", "famille_opération"],
    right_on=["id", "année", "top_famille_opération"]
)
 
top_info = (
    top_details.groupby(["id", "année"]).agg(
        top_type_opération=("type_opération", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
        top_famille_max_volume=("volume_opérations_en_kdh", "max"),
        top_famille_agence=("code_agence", lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    ).reset_index()
)
 
# 4. Diversité des familles
diversity_df = df.groupby(["id", "année"]).agg(
    diversité_opérations=("famille_opération", pd.Series.nunique)
).reset_index()
 
# 5. Opérations vers autres banques
autre_banque_df = df.groupby(["id", "année"]).agg(
    operation_vers_autre_banque=("vers_autre_banque", "any"),
    ops_vers_autre_banque=("vers_autre_banque", "sum"),
    total_ops=("nombre_opérations", "sum")
).reset_index()
autre_banque_df["ratio_ops_vers_autre_banque"] = (
    autre_banque_df["ops_vers_autre_banque"] / autre_banque_df["total_ops"]
)
 
def categoriser_ratio(r):
    if pd.isna(r): return "aucune"
    elif r == 0: return "aucune"
    elif r < 0.1: return "faible"
    elif r < 0.5: return "modéré"
    else: return "élevé"
 
autre_banque_df["cat_ratio_ops_vers_autre_banque"] = autre_banque_df["ratio_ops_vers_autre_banque"].apply(categoriser_ratio)
 
# 6. Fusion intermédiaire
features_df = agg_df.merge(top_famille, on=["id", "année"], how="outer")
features_df = features_df.merge(top_info, on=["id", "année"], how="outer")
features_df = features_df.merge(diversity_df, on=["id", "année"], how="outer")
features_df = features_df.merge(autre_banque_df[[
    "id", "année", "operation_vers_autre_banque",
    "ratio_ops_vers_autre_banque", "cat_ratio_ops_vers_autre_banque"
]], on=["id", "année"], how="outer")
 
# 7. Pivot des années → colonnes
vars_to_pivot = [col for col in features_df.columns if col not in ["id", "année"]]
pivoted = []
for var in vars_to_pivot:
    p = features_df.pivot(index="id", columns="année", values=var)
    p.columns = [f"{var}_{int(c)}" for c in p.columns]
    pivoted.append(p)
 
final_df = pd.concat(pivoted, axis=1).reset_index()
 
final_df.head()

,id,total_operations_2021,total_operations_2022,total_operations_2023,total_operations_2024,active_2021,active_2022,active_2023,active_2024,top_famille_opération_2021,...,operation_vers_autre_banque_2023,operation_vers_autre_banque_2024,ratio_ops_vers_autre_banque_2021,ratio_ops_vers_autre_banque_2022,ratio_ops_vers_autre_banque_2023,ratio_ops_vers_autre_banque_2024,cat_ratio_ops_vers_autre_banque_2021,cat_ratio_ops_vers_autre_banque_2022,cat_ratio_ops_vers_autre_banque_2023,cat_ratio_ops_vers_autre_banque_2024
0,-9,170992.0,168597.0,158343.0,167513.0,True,True,True,True,VERSEMENTS ESPECES,...,False,False,0.000000,0.000000,0.000000,0.000000,aucune,aucune,aucune,aucune
1,-1,29.0,163.0,51.0,36.0,True,True,True,True,VIREMENTS,...,False,False,0.000000,0.000000,0.000000,0.000000,aucune,aucune,aucune,aucune
2,1,6.0,52.0,1.0,NaN,True,True,True,NaN,RETRAITS GAB,...,True,NaN,0.000000,0.019231,1.000000,NaN,aucune,faible,élevé,NaN
3,2,96.0,59.0,15.0,1.0,True,True,True,True,PAIEMENTS CARTE,...,True,False,0.020833,0.016949,0.066667,0.000000,faible,faible,faible,aucune
4,4,414.0,723.0,654.0,579.0,True,True,True,True,RECHARGE MOBILE,...,True,True,0.014493,0.013831,0.012232,0.013817,faible,faible,faible,faible


In [17]:
# Remplissage des valeurs manquantes après le pivot
fillna_dict = {
    # Numériques
    "total_operations": 0,
    "top_famille_total_ops": 0,
    "top_famille_max_volume": 0,
    "diversité_opérations": 0,
    "ratio_ops_vers_autre_banque": 0.0,
    
    # Booléens
    "active": False,
    "operation_vers_autre_banque": False,
 
    # Catégorielles
    "top_famille_opération": "SANS OP",
    "top_type_opération": "SANS OP",
    "top_famille_agence": -1,
    "cat_ratio_ops_vers_autre_banque": "aucune"
}
 
for col_prefix, value in fillna_dict.items():
    cols = [col for col in final_df.columns if col.startswith(col_prefix)]
    if isinstance(value, bool):
        final_df[cols] = final_df[cols].fillna(value).astype(bool)
    else:
        final_df[cols] = final_df[cols].fillna(value)
 
# Affichage ou export
final_df.head()
 

,id,total_operations_2021,total_operations_2022,total_operations_2023,total_operations_2024,active_2021,active_2022,active_2023,active_2024,top_famille_opération_2021,...,operation_vers_autre_banque_2023,operation_vers_autre_banque_2024,ratio_ops_vers_autre_banque_2021,ratio_ops_vers_autre_banque_2022,ratio_ops_vers_autre_banque_2023,ratio_ops_vers_autre_banque_2024,cat_ratio_ops_vers_autre_banque_2021,cat_ratio_ops_vers_autre_banque_2022,cat_ratio_ops_vers_autre_banque_2023,cat_ratio_ops_vers_autre_banque_2024
0,-9,170992.0,168597.0,158343.0,167513.0,True,True,True,True,VERSEMENTS ESPECES,...,False,False,0.000000,0.000000,0.000000,0.000000,aucune,aucune,aucune,aucune
1,-1,29.0,163.0,51.0,36.0,True,True,True,True,VIREMENTS,...,False,False,0.000000,0.000000,0.000000,0.000000,aucune,aucune,aucune,aucune
2,1,6.0,52.0,1.0,0.0,True,True,True,False,RETRAITS GAB,...,True,False,0.000000,0.019231,1.000000,0.000000,aucune,faible,élevé,aucune
3,2,96.0,59.0,15.0,1.0,True,True,True,True,PAIEMENTS CARTE,...,True,False,0.020833,0.016949,0.066667,0.000000,faible,faible,faible,aucune
4,4,414.0,723.0,654.0,579.0,True,True,True,True,RECHARGE MOBILE,...,True,True,0.014493,0.013831,0.012232,0.013817,faible,faible,faible,faible


In [18]:
# 1. Variation du nombre total d'opérations entre les années
if "total_operations_2022" in final_df.columns and "total_operations_2021" in final_df.columns:
    final_df["variation_total_operations_2021_2022"] = final_df.apply(
        lambda row: row["total_operations_2022"] - row["total_operations_2021"]
        if pd.notna(row["total_operations_2022"]) and pd.notna(row["total_operations_2021"])
        else np.nan,
        axis=1
    )
 
if "total_operations_2023" in final_df.columns and "total_operations_2022" in final_df.columns:
    final_df["variation_total_operations_2022_2023"] = final_df.apply(
        lambda row: row["total_operations_2023"] - row["total_operations_2022"]
        if pd.notna(row["total_operations_2023"]) and pd.notna(row["total_operations_2022"])
        else np.nan,
        axis=1
    )
 
if "total_operations_2024" in final_df.columns and "total_operations_2023" in final_df.columns:
    final_df["variation_total_operations_2023_2024"] = final_df.apply(
        lambda row: row["total_operations_2024"] - row["total_operations_2023"]
        if pd.notna(row["total_operations_2024"]) and pd.notna(row["total_operations_2023"])
        else np.nan,
        axis=1
    )
 
# 2. Nombre d'années actives (True = 1, False/NaN = 0)
active_cols = [col for col in final_df.columns if col.startswith("active_") and int(col.split("_")[1]) <= 2023]
final_df["nb_annees_actives"] = final_df[active_cols].fillna(False).astype(int).sum(axis=1)
 
# 3. Évolution du ratio des opérations vers autres banques
if "ratio_ops_vers_autre_banque_2022" in final_df.columns and "ratio_ops_vers_autre_banque_2021" in final_df.columns:
    final_df["evolution_ratio_ops_autre_banque_2022_2021"] = final_df.apply(
        lambda row: row["ratio_ops_vers_autre_banque_2022"] - row["ratio_ops_vers_autre_banque_2021"]
        if pd.notna(row["ratio_ops_vers_autre_banque_2022"]) and pd.notna(row["ratio_ops_vers_autre_banque_2021"])
        else np.nan,
        axis=1
    )
 
if "ratio_ops_vers_autre_banque_2023" in final_df.columns and "ratio_ops_vers_autre_banque_2022" in final_df.columns:
    final_df["evolution_ratio_ops_autre_banque_2023_2022"] = final_df.apply(
        lambda row: row["ratio_ops_vers_autre_banque_2023"] - row["ratio_ops_vers_autre_banque_2022"]
        if pd.notna(row["ratio_ops_vers_autre_banque_2023"]) and pd.notna(row["ratio_ops_vers_autre_banque_2022"])
        else np.nan,
        axis=1
    )
 
if "ratio_ops_vers_autre_banque_2024" in final_df.columns and "ratio_ops_vers_autre_banque_2023" in final_df.columns:
    final_df["evolution_ratio_ops_autre_banque_2024_2023"] = final_df.apply(
        lambda row: row["ratio_ops_vers_autre_banque_2024"] - row["ratio_ops_vers_autre_banque_2023"]
        if pd.notna(row["ratio_ops_vers_autre_banque_2024"]) and pd.notna(row["ratio_ops_vers_autre_banque_2023"])
        else np.nan,
        axis=1
    )

final_df.head()

,id,total_operations_2021,total_operations_2022,total_operations_2023,total_operations_2024,active_2021,active_2022,active_2023,active_2024,top_famille_opération_2021,...,cat_ratio_ops_vers_autre_banque_2022,cat_ratio_ops_vers_autre_banque_2023,cat_ratio_ops_vers_autre_banque_2024,variation_total_operations_2021_2022,variation_total_operations_2022_2023,variation_total_operations_2023_2024,nb_annees_actives,evolution_ratio_ops_autre_banque_2022_2021,evolution_ratio_ops_autre_banque_2023_2022,evolution_ratio_ops_autre_banque_2024_2023
0,-9,170992.0,168597.0,158343.0,167513.0,True,True,True,True,VERSEMENTS ESPECES,...,aucune,aucune,aucune,-2395.0,-10254.0,9170.0,3,0.000000,0.000000,0.000000
1,-1,29.0,163.0,51.0,36.0,True,True,True,True,VIREMENTS,...,aucune,aucune,aucune,134.0,-112.0,-15.0,3,0.000000,0.000000,0.000000
2,1,6.0,52.0,1.0,0.0,True,True,True,False,RETRAITS GAB,...,faible,élevé,aucune,46.0,-51.0,-1.0,3,0.019231,0.980769,-1.000000
3,2,96.0,59.0,15.0,1.0,True,True,True,True,PAIEMENTS CARTE,...,faible,faible,aucune,-37.0,-44.0,-14.0,3,-0.003884,0.049718,-0.066667
4,4,414.0,723.0,654.0,579.0,True,True,True,True,RECHARGE MOBILE,...,faible,faible,faible,309.0,-69.0,-75.0,3,-0.000661,-0.001599,0.001585


In [19]:
# Optionnel : export vers Excel
final_df.to_excel("OpPivoté2.xlsx", index=False)

Nb Connexion

In [77]:
file_path = "C:/Users/mohameds.berrada/Downloads/codespace/test_codespace/data/Nombre de connexions cv.xlsx"
xls = pd.ExcelFile(file_path)
print(xls.sheet_names)

['Tableau1', 'Feuil2', 'Feuil1', '2022', '2023']


In [78]:
df = pd.read_excel(file_path,sheet_name='Tableau1')
df.head()

,ID,Année Connexion,Date Début Relation,Marché,Région,Profession,Âge client,Type Personne,Canal Connexion,CSP,Date Activation,Nombre de connexions,Ligne_Métier
0,2,2022,2015-08-07,MAROCAINS NON RESIDENTS,REGION CASABLANCA-SETTAT,EMPLOYE,47,PHYSIQUE,MBK,EMPLOYE,2021-09-13,94,MRE (hors BP)
1,2,2023,2015-08-07,MAROCAINS NON RESIDENTS,REGION CASABLANCA-SETTAT,EMPLOYE,47,PHYSIQUE,MBK,EMPLOYE,2021-09-13,84,MRE (hors BP)
2,2,2024,2015-08-07,MAROCAINS NON RESIDENTS,REGION CASABLANCA-SETTAT,EMPLOYE,47,PHYSIQUE,MBK,EMPLOYE,2021-09-13,14,MRE (hors BP)
3,4,2022,2015-09-14,PETITE ENTREPRISE,REG TANGER-TETOUAN-HOCEIM,COMMERCANT,40,PHYSIQUE,EBK,COMMERCE BOULANGERIE CAFE,2018-04-15,2,TPE (hors BP)
4,4,2022,2015-09-14,PETITE ENTREPRISE,REG TANGER-TETOUAN-HOCEIM,COMMERCANT,40,PHYSIQUE,MBK,COMMERCE BOULANGERIE CAFE,2018-04-15,813,TPE (hors BP)


In [79]:
# Calcul des features
features = []
 
def categorize_age(age):
    if age < 18: return "<18"
    elif age < 25: return "18-25"
    elif age < 30: return "25-30"
    elif age < 45: return "30-45"
    elif age < 60: return "45-60"
    elif age < 80: return "60-80"
    else: return "80>"
 
for (client_id, year), group in df.groupby(["ID", "Année Connexion"]):
    total_conn = group["Nombre de connexions"].sum()
    mean_conn = group["Nombre de connexions"].mean()
 
    top_canal = group.groupby("Canal Connexion")["Nombre de connexions"].sum().sort_values(ascending=False)
    top_canal_name = top_canal.index[0]
    top_canal_count = top_canal.iloc[0]
 
    prev_year = year - 1
    prev_conn = df[(df["ID"] == client_id) & (df["Année Connexion"] == prev_year)]["Nombre de connexions"].sum()
 
    if total_conn > prev_conn:
        tendance = "en hausse"
    elif total_conn < prev_conn:
        tendance = "en baisse"
    else:
        tendance = "stable"
 
    global_mean = df[df["Année Connexion"] == year]["Nombre de connexions"].mean()
    actif_digital = mean_conn > global_mean
 
    row_static = df[df["ID"] == client_id].iloc[0]
    age = row_static["Âge client"]
    age_cat = categorize_age(age)
    debut_relation = df[df["ID"] == client_id]['Date Début Relation'].min()
    debut_relation = pd.to_datetime(debut_relation).date()
 
    features.append({
        "ID": client_id,
        "Année": year,
        "Date Début Relation": debut_relation,
        "nb_connexions": total_conn,
        "moyenne_connexions": mean_conn,
        "top_canal": top_canal_name,
        "nb_top_canal": top_canal_count,
        "tendance_connexions": tendance,
        "plus_actif_que_moyenne": actif_digital,
        "Âge client": age,
        "catégorie_âge": age_cat,
        "CSP": row_static["CSP"],
        "Ligne_Métier": row_static["Ligne_Métier"],
        "Type Personne": row_static["Type Personne"],
        "canal_unique": group["Canal Connexion"].nunique() == 1
    })
 
df_feat = pd.DataFrame(features)

# Calcul des seuils globaux (par année possible)
quantile_75 = df.groupby("Année Connexion")["Nombre de connexions"].quantile(0.75).to_dict()
 
# Ajout du type_utilisateur sans utiliser fidélité
for i, row in df_feat.iterrows():
    year = row["Année"]
    nb_conn = row["nb_connexions"]
    actif = row["plus_actif_que_moyenne"]
    tendance = row["tendance_connexions"]
 
    if actif and nb_conn >= quantile_75.get(year, 0):
        type_user = "digital intensif" #actif digital et nb_connexion >= 75e percentile année
    elif actif:
        type_user = "digital modéré" #actif digital mais nb_connexion < 75e percentile année
    elif not actif and tendance == "en baisse":
        type_user = "en perte de vitesse"
    else:
        type_user = "occasionnel"

    df_feat.at[i,"type_utilisateur"] = type_user

# Liste des colonnes dynamiques à pivoter par année
cols_to_pivot = [
    "nb_connexions",
    "moyenne_connexions",
    "top_canal",
    "nb_top_canal",
    "tendance_connexions",
    "plus_actif_que_moyenne",
    "type_utilisateur"
]
 
# Pivot des colonnes dynamiques
df_pivot = df_feat.pivot(index="ID", columns="Année", values=cols_to_pivot)
 
# Aplatir les multi-index de colonnes
df_pivot.columns = [f"{col}_{year}" for col, year in df_pivot.columns]

# === Étape 7 : Remplissage des valeurs manquantes ===
fill_values = {}
 
# Valeurs numériques
for col in df_pivot.columns:
    if "nb_connexions" in col or "moyenne_connexions" in col or "nb_top_canal" in col:
        fill_values[col] = 0
    elif "plus_actif_que_moyenne" in col:
        fill_values[col] = False
    elif "top_canal" in col:
        fill_values[col] = "SANS CANAL"
    elif "tendance_connexions" in col:
        fill_values[col] = "inconnue"
    elif "type_utilisateur" in col:
        fill_values[col] = "occasionnel"
 
# Remplir les NA
df_pivot.fillna(value=fill_values, inplace=True)
 
# Récupération des colonnes statiques depuis la dernière apparition de chaque client
static_cols = [
    "Âge client",
    "catégorie_âge",
    "CSP",
    "Ligne_Métier",
    "Type Personne",
    "canal_unique",
    "Date Début Relation"
]
df_static = df_feat.sort_values("Année").drop_duplicates("ID", keep="last")[["ID"] + static_cols].set_index("ID")
 
# Concaténation le long des colonnes (axis=1)
df_final_pivot = pd.concat([df_static, df_pivot], axis=1).reset_index()
 
# Résultat
df_final_pivot.head()

,ID,Âge client,catégorie_âge,CSP,Ligne_Métier,Type Personne,canal_unique,Date Début Relation,nb_connexions_2022,nb_connexions_2023,...,tendance_connexions_2024,tendance_connexions_2025,plus_actif_que_moyenne_2022,plus_actif_que_moyenne_2023,plus_actif_que_moyenne_2024,plus_actif_que_moyenne_2025,type_utilisateur_2022,type_utilisateur_2023,type_utilisateur_2024,type_utilisateur_2025
0,2,47,45-60,EMPLOYE,MRE (hors BP),PHYSIQUE,True,2015-08-07,94,84,...,en baisse,inconnue,False,False,False,False,occasionnel,en perte de vitesse,en perte de vitesse,occasionnel
1,4,40,30-45,COMMERCE BOULANGERIE CAFE,TPE (hors BP),PHYSIQUE,False,2015-09-14,815,667,...,en baisse,en baisse,True,False,False,False,digital intensif,en perte de vitesse,en perte de vitesse,en perte de vitesse
2,5,48,45-60,OUVRIER,MRE (hors BP),PHYSIQUE,True,2015-08-08,23,0,...,en hausse,en baisse,False,False,False,False,occasionnel,occasionnel,occasionnel,en perte de vitesse
3,7,60,60-80,SANS REVENU NON MINEUR,Particuliers & Professionnels (hors BP),PHYSIQUE,True,2015-08-10,138,161,...,en hausse,en baisse,False,False,False,True,occasionnel,occasionnel,occasionnel,digital modéré
4,16,55,45-60,EMPLOYE,Particuliers & Professionnels (hors BP),PHYSIQUE,True,2015-08-17,41,65,...,en hausse,en baisse,False,False,False,False,occasionnel,occasionnel,occasionnel,en perte de vitesse


In [80]:
# Sauvegarde optionnelle
df_final_pivot.to_excel("clients_transformés2.xlsx", index=False)

CMC

In [3]:
file_path = "PATH/data/STATUT CLIENT CMC CMD SOLDE cv.xls"
xls = pd.ExcelFile(file_path)
print(xls.sheet_names)

['Feuil1']


In [4]:
df = pd.read_excel(file_path,sheet_name='Feuil1')
df.head()

,Année,Mois,Profession,CSP,Secteur d'activité,ID,Statut Attrition,Solde à M (Dh),Mouvements Cumulés Créditeurs à M (Dh),Mouvements Cumulés Débiteurs à M (Dh)
0,2022,12,RENTIER PROPRIETAIRE DE BIENS,RENTIER,NON RENSEIGNE,1721,CLIENT EN GEL,94.43,12369.00,15924.09000
1,2022,12,RENTIER PROPRIETAIRE DE BIENS,RENTIER,NON RENSEIGNE,1836,CLIENT EN GEL,231.42,1109.22,1143.00333
2,2022,12,RENTIER PROPRIETAIRE DE BIENS,RENTIER,NON RENSEIGNE,2467,CLIENT EN ATTRITION,-33.25,17290.00,20720.07000
3,2022,12,RENTIER PROPRIETAIRE DE BIENS,RENTIER,NON RENSEIGNE,2745,CLIENT EN GEL,795.34,228644.29,228017.86000
4,2022,12,RENTIER PROPRIETAIRE DE BIENS,RENTIER,NON RENSEIGNE,2751,CLIENT EN ATTRITION,129.01,90619.55,94532.41000


In [ ]:
# Nettoyer les colonnes
df.columns = df.columns.str.strip()
 
# Définir les colonnes numériques à traiter
numeric_cols = [
    'Solde à M (Dh)',
    'Mouvements Cumulés Créditeurs à M (Dh)',
    'Mouvements Cumulés Débiteurs à M (Dh)'
]
 
# Nettoyage des valeurs numériques
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
 
# Marquage de présence
df['Présence'] = 1
 
# Liste unique des années et clients
clients = df[['ID', 'Profession', 'CSP', "Secteur d'activité"]].drop_duplicates()
annees = df['Année'].dropna().unique()
annees.sort()
 
# Produit cartésien clients × années
full_index = pd.MultiIndex.from_product([clients['ID'], annees], names=['ID', 'Année'])
full_df = pd.DataFrame(index=full_index).reset_index()
full_df = full_df.merge(clients, on='ID', how='left')
 
# Fusion avec les données originales
df_full = full_df.merge(df, on=['ID', 'Année'], how='left', suffixes=('', '_orig'))
 
# Remplir les valeurs d'activité manquantes par 0
df_full[numeric_cols] = df_full[numeric_cols].fillna(0)
df_full['Présence'] = df_full['Présence'].fillna(0)
 
# Pivot des colonnes numériques
pivoted = df_full.pivot_table(
    index=['ID', 'Profession', 'CSP', "Secteur d'activité"],
    columns='Année',
    values=numeric_cols + ['Présence'],
    aggfunc='sum',
    fill_value=0
)
 
# Aplatir les colonnes
pivoted.columns = [f"{col[0].replace(' ', '_')}_{col[1]}" for col in pivoted.columns]
pivoted.reset_index(inplace=True)
 
# Ajouter des variables dérivées
def safe_diff(col_prefix, year1, year2):
    col_y1 = f"{col_prefix}_{year1}"
    col_y2 = f"{col_prefix}_{year2}"
    if col_y1 in pivoted.columns and col_y2 in pivoted.columns:
        pivoted[f"Variation_{col_prefix}_{year2}_{year1}"] = pivoted[col_y2] - pivoted[col_y1]
 
safe_diff('Solde_à_M_(Dh)', 2022, 2023)
safe_diff('Solde_à_M_(Dh)', 2023, 2024)
safe_diff('Mouvements_Cumulés_Créditeurs_à_M_(Dh)', 2022, 2023)
#safe_diff('Mouvements_Cumulés_Créditeurs_à_M_(Dh)', 2023, 2024)
safe_diff('Mouvements_Cumulés_Débiteurs_à_M_(Dh)', 2022, 2023)
#safe_diff('Mouvements_Cumulés_Débiteurs_à_M_(Dh)', 2023, 2024)

# Création d’indicateurs sur solde négatif
for year in [2022, 2023]:
    col = f"Solde_à_M_(Dh)_{year}"
    if col in pivoted.columns:
        pivoted[f"Solde_négatif_{year}"] = pivoted[col] < 0
 
# Nombre d'années avec solde négatif
solde_cols = [f"Solde_à_M_(Dh)_{year}" for year in [2022, 2023] if f"Solde_à_M_(Dh)_{year}" in pivoted.columns]
pivoted['Nb_Années_Solde_Négatif'] = pivoted[solde_cols].lt(0).sum(axis=1)
 
# Solde minimal observé (gravité)
pivoted['Solde_min'] = pivoted[solde_cols].min(axis=1)


pivoted.head()

,ID,Profession,CSP,Secteur d'activité,Mouvements_Cumulés_Créditeurs_à_M_(Dh)_2022,Mouvements_Cumulés_Créditeurs_à_M_(Dh)_2023,Mouvements_Cumulés_Créditeurs_à_M_(Dh)_2024,Mouvements_Cumulés_Débiteurs_à_M_(Dh)_2022,Mouvements_Cumulés_Débiteurs_à_M_(Dh)_2023,Mouvements_Cumulés_Débiteurs_à_M_(Dh)_2024,Présence_2022,Présence_2023,Présence_2024,Solde_à_M_(Dh)_2022,Solde_à_M_(Dh)_2023,Solde_à_M_(Dh)_2024,Solde_négatif_2022,Nb_Années_Solde_Négatif,Solde_min
0,1,RENTIER,RENTIER,PROFESSIONS LIBERALES DIVERSES,52003.00,0.00,0.00,59532.13,33.25,0.00,1,1,1,2532.32,-444.22,-444.22,False,0,2532.32
1,2,EMPLOYE,EMPLOYE,NON RENSEIGNE,18753.00,6118.00,0.00,26026.77,5449.01,371.07,1,1,1,671.65,1336.65,964.25,False,0,671.65
2,3,TAILLEUR,ARTISANAT METIER INDEPEND,COMMERCE DE DETAIL DES TISSUS,0.00,0.00,0.00,371.07,131511.73,67.83,1,1,1,129675.00,-1907.22,-2122.68,False,0,129675.00
3,4,COMMERCANT,COMMERCE BOULANGERIE CAFE,COMMERCE DE DETAIL D ALIMENTATION GENERALE,6481987.75,3364560.85,3561563.11,8459450.37,3556401.38,4216375.31,1,1,1,942287.71,750251.67,95432.82,False,0,942287.71
4,5,NON RENSEIGNE,OUVRIER,ACTIVITES EXERCEES HORS DU TERRITOIRE MAROCAIN,170016.56,0.00,98034.30,275094.54,319.20,58496.06,1,1,1,434.91,114.38,39627.35,False,0,434.91


In [ ]:
# Export final
pivoted.to_excel("CmcPivoté2.xlsx", index=False)
print("Fichier exporté avec succès.")

Fichier exporté avec succès.


Base Attrition

In [2]:
file_path = "C:/Users/mohameds.berrada/Downloads/codespace/test_codespace/data/Base clients pour analyse attrition cv.xlsx"
xls = pd.ExcelFile(file_path)
print(xls.sheet_names)

['Base stage']


In [3]:
df = pd.read_excel(file_path,sheet_name='Base stage')
df.head()

,Année,Année entrée en relation du client,ID client,Ligne Métier,Mois (#),Niveau de Service,Niveau de Service Ancien,Encours Moyen
0,2020,2009,6383,Particuliers & Professionnels (hors BP),1,S2 - PREMIUM,CLUB,40367.3753
1,2020,2009,6383,Particuliers & Professionnels (hors BP),2,S2 - PREMIUM,CLUB,997197.2787
2,2020,2009,6383,Particuliers & Professionnels (hors BP),3,S2 - PREMIUM,CLUB,432950.0056
3,2020,2009,6383,Particuliers & Professionnels (hors BP),4,S2 - PREMIUM,CLUB,250954.1755
4,2020,2009,6383,Particuliers & Professionnels (hors BP),5,S2 - PREMIUM,CLUB,175862.3888


In [4]:
def regrouper_niveau_service(niveau):
    if niveau in ['S1 - EXCELLENCE', 'S2-PME SUPERIEURE','S2 - PREMIUM']:
        return 'Affluent'
    elif niveau in ['G1 - GD', 'G2 - GD','G3 - GD','G4 - GD','G5 - GD','CONTENTIEUX','GD GD > 12 M','I4-INACTIF SS POTEN','I3-INACTIF A POTENT','GE']:
        return 'Gelé inactif'
    elif niveau in ['S3 - POTENTIEL', 'S4 - AVENIR', 'S5 - UNIVERSEL', 'S4-MASS PME', 'S3-PME INTERMEDIAIRE','S6 - LIB']:
        return 'Mass'
    else:
        return niveau  # Pour conserver les autres niveaux s'il y en a
 
# Appliquer la fonction de regroupement à la colonne 'Niveau de Service'
df['Niveau_Service_Regroupe'] = df['Niveau de Service'].apply(regrouper_niveau_service)
 
# Afficher les valeurs uniques de l'ancienne et de la nouvelle colonne pour vérifier
print("Valeurs uniques de 'Niveau de Service' avant regroupement:")
print(df['Niveau de Service'].unique())
 
print("\nValeurs uniques de 'Niveau_Service_Regroupe' après regroupement:")
print(df['Niveau_Service_Regroupe'].unique())

Valeurs uniques de 'Niveau de Service' avant regroupement:
['S2 - PREMIUM' 'S3 - POTENTIEL' 'S4 - AVENIR' 'S5 - UNIVERSEL'
 'GD GD > 12 M' 'NON RENSEIGNE' 'S1 - EXCELLENCE' 'G4 - GD' 'S6 - LIB'
 'S2-PME SUPERIEURE' 'G5 - GD' 'G1 - GD' 'G3 - GD' 'S4-MASS PME'
 'S3-PME INTERMEDIAIRE' 'CONTENTIEUX' 'I4-INACTIF SS POTEN' 'G2 - GD'
 'I3-INACTIF A POTENT' 'NOUVEAU CLIENT' 'GE']

Valeurs uniques de 'Niveau_Service_Regroupe' après regroupement:
['Affluent' 'Mass' 'Gelé inactif' 'NON RENSEIGNE' 'NOUVEAU CLIENT']


In [ ]:
from itertools import product
  
# Convertir la colonne Date si ce n’est pas fait
df['Date'] = pd.to_datetime(dict(year=df['Année'], month=df['Mois (#)'], day=1))
 
# Complétion des mois manquants
clients = df['ID client'].unique()
mois_complets = pd.date_range(start='2020-01-01', end='2024-12-01', freq='MS')
full_grid = pd.DataFrame(product(clients, mois_complets), columns=['ID client', 'Date'])
full_grid['Année'] = full_grid['Date'].dt.year
full_grid['Mois (#)'] = full_grid['Date'].dt.month
 
df = pd.merge(full_grid, df, on=['ID client', 'Année', 'Mois (#)', 'Date'], how='left')
df['Encours Moyen'] = df['Encours Moyen'].fillna(0)
 
# Encours moyen cumulé par année
emc = df.groupby(['ID client', 'Année'])['Encours Moyen'].mean().unstack().add_prefix('EMC_').fillna(0)
 
# Évolution EM par année
evolutions = {}
years = sorted(df['Année'].unique())
for i in range(1, len(years)):
    y1, y2 = years[i-1], years[i]
    col = f"Evolution_EM_{y2}vs{y1}"
    evol = ((emc[f'EMC_{y2}'] - emc[f'EMC_{y1}']) / (emc[f'EMC_{y1}'] + 1e-6)) * 100
    evol = evol.clip(upper=101)
    evolutions[col] = evol
evol_df = pd.DataFrame(evolutions)

# Feature : nombre de mois avec EM > 0 par an
mois_actifs = df[df['Encours Moyen'] > 0].groupby(['ID client', 'Année'])['Mois (#)'].count().unstack().add_prefix('MoisActifs_').fillna(0)

# Volatilite de l'EM
volatilite_202X = df[df['Année']<=2024].groupby(['ID client', 'Année'])['Encours Moyen'].std().unstack().add_prefix('Volatilite_EM_').fillna(0)
 
# Nb de mois consécutifs avec EM=0 par année
def max_consecutive_zeros(group):
    group = group.sort_values('Date')
    result = {}
    for year, data in group.groupby('Année'):
        streak = max_streak = 0
        for em in data['Encours Moyen']:
            if em == 0:
                streak += 1
                max_streak = max(max_streak, streak)
            else:
                streak = 0
        result[f"ZeroStreak_{year}"] = max_streak
    return pd.Series(result)
 
zero_streaks = df.groupby('ID client').apply(max_consecutive_zeros).fillna(0)
 
# Différence entre 1er et dernier EM par année
def diff_first_last(group):
    group = group.sort_values('Date')
    result = {}
    for year, data in group.groupby('Année'):
        if len(data) >= 1:
            diff = data['Encours Moyen'].iloc[-1] - data['Encours Moyen'].iloc[0]
        else:
            diff = 0
        result[f"Delta_EM_{year}"]= diff
    return pd.Series(result)
 
delta_em = df.groupby('ID client').apply(diff_first_last).fillna(0)

# Récupération des colonnes statiques depuis la dernière apparition de chaque client
static_cols = [
    "Ligne Métier",
    "Niveau_Service_Regroupe"
]
df_static = df.groupby("ID client")[static_cols].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
 
# Fusion de toutes les features
dataset = pd.concat([emc, evol_df, zero_streaks, delta_em, mois_actifs, volatilite_202X, df_static], axis=1).reset_index()

dataset.head()

,ID client,EMC_2020,EMC_2021,EMC_2022,EMC_2023,EMC_2024,Evolution_EM_2021vs2020,Evolution_EM_2022vs2021,Evolution_EM_2023vs2022,Evolution_EM_2024vs2023,...,MoisActifs_2022,MoisActifs_2023,MoisActifs_2024,Volatilite_EM_2020,Volatilite_EM_2021,Volatilite_EM_2022,Volatilite_EM_2023,Volatilite_EM_2024,Ligne Métier,Niveau_Service_Regroupe
0,1,5.195431e+03,5.190141e+03,6.123909e+03,69.068008,7.980000,99.898178,101.000000,1.127842,11.553829,...,12.0,12.0,12.0,3039.731707,3535.314801,2.070158e+03,211.615068,0.000000,Particuliers & Professionnels (hors BP),Mass
1,2,6.144167e+03,1.122189e+04,2.206286e+03,1219.975750,1252.103008,101.000000,19.660550,55.295449,101.000000,...,12.0,12.0,12.0,6048.074606,13457.091284,2.487441e+03,525.045669,145.283095,MRE (hors BP),Affluent
2,3,1.323133e+05,1.319292e+05,1.315489e+05,77264.360542,0.000000,99.709684,99.711763,58.734327,0.000000,...,12.0,8.0,0.0,112.973196,114.476288,1.157234e+02,66749.278765,0.000000,TPE (hors BP),Gelé inactif
3,4,1.220724e+06,1.955356e+06,2.108484e+06,417360.774133,603263.832792,101.000000,101.000000,19.794352,101.000000,...,12.0,12.0,12.0,198449.686505,453797.362144,1.234533e+06,369574.570043,427327.801590,TPE (hors BP),Affluent
4,5,8.007897e+04,1.057855e+05,5.328427e+04,364.657183,14809.650858,101.000000,50.370086,0.684362,101.000000,...,12.0,12.0,12.0,15093.089524,130.795422,6.296749e+04,130.795422,27991.043083,MRE (hors BP),Mass


In [6]:
for annee in [2020, 2021, 2022, 2023, 2024]:  # adapte selon les années disponibles
    col = f'Volatilite_EM_{annee}'
    if col in dataset.columns:
        mean = dataset[col].mean()
        std = dataset[col].std()
 
        def classer_volatilite(x):
            if x < mean - std:
                return 'Basse - Stable'
            elif x > mean + std:
                return 'Haute - Instable'
            else:
                return 'Moyenne'
 
        dataset[f'Categorie_Volatilite_{annee}'] = dataset[col].apply(classer_volatilite)

dataset.head()

,ID client,EMC_2020,EMC_2021,EMC_2022,EMC_2023,EMC_2024,Evolution_EM_2021vs2020,Evolution_EM_2022vs2021,Evolution_EM_2023vs2022,Evolution_EM_2024vs2023,...,Volatilite_EM_2022,Volatilite_EM_2023,Volatilite_EM_2024,Ligne Métier,Niveau_Service_Regroupe,Categorie_Volatilite_2020,Categorie_Volatilite_2021,Categorie_Volatilite_2022,Categorie_Volatilite_2023,Categorie_Volatilite_2024
0,1,5.195431e+03,5.190141e+03,6.123909e+03,69.068008,7.980000,99.898178,101.000000,1.127842,11.553829,...,2.070158e+03,211.615068,0.000000,Particuliers & Professionnels (hors BP),Mass,Moyenne,Moyenne,Moyenne,Moyenne,Moyenne
1,2,6.144167e+03,1.122189e+04,2.206286e+03,1219.975750,1252.103008,101.000000,19.660550,55.295449,101.000000,...,2.487441e+03,525.045669,145.283095,MRE (hors BP),Affluent,Moyenne,Moyenne,Moyenne,Moyenne,Moyenne
2,3,1.323133e+05,1.319292e+05,1.315489e+05,77264.360542,0.000000,99.709684,99.711763,58.734327,0.000000,...,1.157234e+02,66749.278765,0.000000,TPE (hors BP),Gelé inactif,Moyenne,Moyenne,Moyenne,Moyenne,Moyenne
3,4,1.220724e+06,1.955356e+06,2.108484e+06,417360.774133,603263.832792,101.000000,101.000000,19.794352,101.000000,...,1.234533e+06,369574.570043,427327.801590,TPE (hors BP),Affluent,Haute - Instable,Haute - Instable,Haute - Instable,Haute - Instable,Haute - Instable
4,5,8.007897e+04,1.057855e+05,5.328427e+04,364.657183,14809.650858,101.000000,50.370086,0.684362,101.000000,...,6.296749e+04,130.795422,27991.043083,MRE (hors BP),Mass,Moyenne,Moyenne,Moyenne,Moyenne,Moyenne


In [7]:
df_test = dataset.copy()
df_test.head()

,ID client,EMC_2020,EMC_2021,EMC_2022,EMC_2023,EMC_2024,Evolution_EM_2021vs2020,Evolution_EM_2022vs2021,Evolution_EM_2023vs2022,Evolution_EM_2024vs2023,...,Volatilite_EM_2022,Volatilite_EM_2023,Volatilite_EM_2024,Ligne Métier,Niveau_Service_Regroupe,Categorie_Volatilite_2020,Categorie_Volatilite_2021,Categorie_Volatilite_2022,Categorie_Volatilite_2023,Categorie_Volatilite_2024
0,1,5.195431e+03,5.190141e+03,6.123909e+03,69.068008,7.980000,99.898178,101.000000,1.127842,11.553829,...,2.070158e+03,211.615068,0.000000,Particuliers & Professionnels (hors BP),Mass,Moyenne,Moyenne,Moyenne,Moyenne,Moyenne
1,2,6.144167e+03,1.122189e+04,2.206286e+03,1219.975750,1252.103008,101.000000,19.660550,55.295449,101.000000,...,2.487441e+03,525.045669,145.283095,MRE (hors BP),Affluent,Moyenne,Moyenne,Moyenne,Moyenne,Moyenne
2,3,1.323133e+05,1.319292e+05,1.315489e+05,77264.360542,0.000000,99.709684,99.711763,58.734327,0.000000,...,1.157234e+02,66749.278765,0.000000,TPE (hors BP),Gelé inactif,Moyenne,Moyenne,Moyenne,Moyenne,Moyenne
3,4,1.220724e+06,1.955356e+06,2.108484e+06,417360.774133,603263.832792,101.000000,101.000000,19.794352,101.000000,...,1.234533e+06,369574.570043,427327.801590,TPE (hors BP),Affluent,Haute - Instable,Haute - Instable,Haute - Instable,Haute - Instable,Haute - Instable
4,5,8.007897e+04,1.057855e+05,5.328427e+04,364.657183,14809.650858,101.000000,50.370086,0.684362,101.000000,...,6.296749e+04,130.795422,27991.043083,MRE (hors BP),Mass,Moyenne,Moyenne,Moyenne,Moyenne,Moyenne


In [8]:
for col in df_test.columns:
    if col.startswith('Evolution_EM_'):
        annee = col.split('_')[2][:4]  # Extrait l'année de début, par exemple "2021" dans "Evolution_EM_2021vs2022"
        attr_col = f'Attrition_{int(annee)}'  # L'attrition concerne l'année suivante
        df_test[attr_col] = (df_test[col] <= -90).astype(int)

In [75]:
df_test.head()

,ID client,EMC_2020,EMC_2021,EMC_2022,EMC_2023,EMC_2024,Evolution_EM_2021vs2020,Evolution_EM_2022vs2021,Evolution_EM_2023vs2022,Evolution_EM_2024vs2023,...,Niveau_Service_Regroupe,Categorie_Volatilite_2020,Categorie_Volatilite_2021,Categorie_Volatilite_2022,Categorie_Volatilite_2023,Categorie_Volatilite_2024,Attrition_2021,Attrition_2022,Attrition_2023,Attrition_2024
0,1,5.195431e+03,5.190141e+03,6.123909e+03,69.068008,7.980000,-0.101822,17.991178,-98.872158,-88.446169,...,Mass,Moyenne,Moyenne,Moyenne,Moyenne,Moyenne,0,0,1,0
1,2,6.144167e+03,1.122189e+04,2.206286e+03,1219.975750,1252.103008,82.643058,-80.339450,-44.704551,2.633434,...,Affluent,Moyenne,Moyenne,Moyenne,Moyenne,Moyenne,0,0,0,0
2,3,1.323133e+05,1.319292e+05,1.315489e+05,77264.360542,0.000000,-0.290316,-0.288237,-41.265673,-100.000000,...,Gelé inactif,Moyenne,Moyenne,Moyenne,Moyenne,Moyenne,0,0,0,1
3,4,1.220724e+06,1.955356e+06,2.108484e+06,417360.774133,603263.832792,60.180032,7.831186,-80.205648,44.542533,...,Affluent,Haute - Instable,Haute - Instable,Haute - Instable,Haute - Instable,Haute - Instable,0,0,0,0
4,5,8.007897e+04,1.057855e+05,5.328427e+04,364.657183,14809.650858,32.101520,-49.629914,-99.315638,101.000000,...,Mass,Moyenne,Moyenne,Moyenne,Moyenne,Moyenne,0,0,1,0


In [9]:
# Export final
df_test.to_excel("ClientsAttrition202X.xlsx", index=False)
print("Fichier exporté avec succès.")

Fichier exporté avec succès.


In [37]:
del df_test
print(df_test)

NameError: name 'df_test' is not defined

In [ ]:
# 9. (Optionnel) Réorganiser les colonnes
#colonnes_finales = ['ID client', 'Année', 'Mois (#)', 'Date', 'Encours Moyen']
#autres = [col for col in df_complet.columns if col not in colonnes_finales]
#df_complet = df_complet[colonnes_finales + autres]
#df_complet.head()

#df["Année entrée en relation du client"] = df["Année entrée en relation du client"].replace("-", 1900).astype("Int64")
#df["Année entrée en relation du client"] = pd.to_numeric(df["Année entrée en relation du client"])
#print(df["Année entrée en relation du client"].unique())